### Visualize results of Neural Non-rigid traking using jupyter notebook

In [1]:
import os
import cv2
import time
import numpy as np
import ipywidgets
import ipyvolume
import json
import torch
import open3d as o3d
import matplotlib.pyplot as plt
import imageio
from sklearn.neighbors import KDTree


# Modules 
from utils import image_proc
from model.model import DeformNet
from model import dataset

import utils.utils as utils
import utils.viz_utils as viz_utils
import utils.nnutils as nnutils
import utils.line_mesh as line_mesh_utils
import options as opt


# We will overwrite the default value in options.py / settings.py
opt.use_mask = True
o3d.visualization.RenderOption.line_width=160.0  


In [5]:
class ExampleSelector():
    def __init__(self,jsonfile="../DeepDeform/test_graphs.json"):
        with open(jsonfile,'r') as f:
            self.examples = json.load(f)
        self.example = None
        self.exampledropdown = ipywidgets.Dropdown(\
            options=[f"{x['seq_id']}-{x['object_id']}-{x['source_id']}-{x['target_id']}"\
                     for x in self.examples],\
            description="Example:")

        self.exampledropdown.observe(self.on_change)
        
        display(self.exampledropdown)
        
        self.split = jsonfile.split('/')[-1].split('_')[0]
        
        #####################################################################################################
        # Load model
        #####################################################################################################

        saved_model = opt.saved_model

        assert os.path.isfile(saved_model), f"Model {saved_model} does not exist."
        pretrained_dict = torch.load(saved_model)

        # Construct model
        self.model = DeformNet().cuda()

        if "chairs_things" in saved_model:
            self.model.flow_net.load_state_dict(pretrained_dict)
        else:
            if opt.model_module_to_load == "full_model":
                # Load completely model            
                self.model.load_state_dict(pretrained_dict)
            elif opt.model_module_to_load == "only_flow_net":
                # Load only optical flow part
                self.model_dict = model.state_dict()
                # 1. filter out unnecessary keys
                pretrained_dict = {k: v for k, v in pretrained_dict.items() if "flow_net" in k}
                # 2. overwrite entries in the existing state dict
                self.model_dict.update(pretrained_dict) 
                # 3. load the new state dict
                self.model.load_state_dict(model_dict)
            else:
                print(opt.model_module_to_load, "is not a valid argument (A: 'full_model', B: 'only_flow_net')")
                exit()

        self.model.eval()
        
        
    def on_change(self,change):
        if change['type'] == 'change' and change['name'] == 'value':
            self.image_list = []
            self.example = self.examples[self.exampledropdown.index]
            for i in range(1,300,5):
                if i%10 == 1: print(f"{i}/100 Completed")
                source_ind = int(self.example['source_color'].split('/')[-1].split('.')[0])
                self.example['target_color'] = "/".join(self.example['target_color'].split('/')[:-1]) + '/' + str(source_ind + i).zfill(6) + '.jpg'
                
                self.example['target_depth'] = "/".join(self.example['target_depth'].split('/')[:-1]) + '/' + str(source_ind + i).zfill(6) + '.png'
                print(self.example['target_color'],self.example['target_depth'])
                try: 
                    self.load_example()
                    break
                except Exception as e:
                    print("Error:",e)
                    continue
            imageio.mimsave('movie.gif', self.image_list)
    def load_example(self):
        
        global image_list
        split = "test"
        seq_id = self.example['seq_id']
        src_id = self.example['source_id'] # source frame
        tgt_id = self.example['target_id'] # target frame
        
        src_color_image_path         = os.path.join("../DeepDeform/" , self.example['source_color'])
        src_depth_image_path         = os.path.join("../DeepDeform/" , self.example['source_depth'])
        tgt_color_image_path         = os.path.join("../DeepDeform/" , self.example['target_color'])
        tgt_depth_image_path         = os.path.join("../DeepDeform/" , self.example['target_depth'])
        graph_nodes_path             = os.path.join("../DeepDeform/" , self.example["graph_nodes"])
        graph_edges_path             = os.path.join("../DeepDeform/" , self.example["graph_edges"])
        graph_edges_weights_path     = os.path.join("../DeepDeform/" , self.example["graph_edges_weights"])
        graph_clusters_path          = os.path.join("../DeepDeform/" , self.example["graph_clusters"])
        pixel_anchors_path           = os.path.join("../DeepDeform/" , self.example["pixel_anchors"])
        pixel_weights_path           = os.path.join("../DeepDeform/" , self.example["pixel_weights"])

        # Make sure to use the intrinsics corresponding to the seq_id above!!  
        intrinsics = {
            "fx": 575.548,
            "fy": 577.46,
            "cx": 323.172,
            "cy": 236.417
        }

        # Some params for coloring the predicted correspondence confidences
        weight_thr = 0.3
        weight_scale = 1


        image_height = opt.image_height
        image_width  = opt.image_width
        max_boundary_dist = opt.max_boundary_dist

        
        
        # Source color and depth
        source, _, cropper = dataset.DeformDataset.load_image(
            src_color_image_path, src_depth_image_path, intrinsics, image_height, image_width
        )

        # Target color and depth (and boundary mask)
        target, target_boundary_mask, _ = dataset.DeformDataset.load_image(
            tgt_color_image_path, tgt_depth_image_path, intrinsics, image_height, image_width, cropper=cropper,
            max_boundary_dist=max_boundary_dist, compute_boundary_mask=True
        )

        # Graph
        graph_nodes, graph_edges, graph_edges_weights, _, graph_clusters, pixel_anchors, pixel_weights = dataset.DeformDataset.load_graph_data(
            graph_nodes_path, graph_edges_path, graph_edges_weights_path, None, 
            graph_clusters_path, pixel_anchors_path, pixel_weights_path, cropper
        )
        
        print("Graph Nodes",graph_nodes.shape)
        print(graph_nodes)
        print("Graph Edges",graph_edges.shape)
        print(graph_edges)
        print("Graph Edge Weights",graph_edges_weights.shape)
        print(graph_edges_weights)
        print(graph_edges_weights[0,0], np.linalg.norm(graph_nodes[0] - graph_nodes[graph_edges[0,0]],ord=1))
        print("Graph Clusters",graph_clusters.shape)
        print(graph_clusters)
        print("Pixel Anchors",pixel_anchors.shape)
        print(pixel_anchors)
        print("Pixel Anchors",pixel_weights.shape)
        print(pixel_weights)
        
        break

        
        num_nodes = np.array(graph_nodes.shape[0], dtype=np.int64)
                
        # Update intrinsics to reflect the crops
        fx, fy, cx, cy = image_proc.modify_intrinsics_due_to_cropping(
            intrinsics['fx'], intrinsics['fy'], intrinsics['cx'], intrinsics['cy'], 
            image_height, image_width, original_h=cropper.h, original_w=cropper.w
        )

        intrinsics = np.zeros((4), dtype=np.float32)
        intrinsics[0] = fx
        intrinsics[1] = fy
        intrinsics[2] = cx
        intrinsics[3] = cy
        

        #####################################################################################################
        # Predict deformation
        #####################################################################################################

        # Move to device and unsqueeze in the batch dimension (to have batch size 1)
        source_cuda               = torch.from_numpy(source).cuda().unsqueeze(0)
        target_cuda               = torch.from_numpy(target).cuda().unsqueeze(0)
        target_boundary_mask_cuda = torch.from_numpy(target_boundary_mask).cuda().unsqueeze(0)
        graph_nodes_cuda          = torch.from_numpy(graph_nodes).cuda().unsqueeze(0)
        graph_edges_cuda          = torch.from_numpy(graph_edges).cuda().unsqueeze(0)
        graph_edges_weights_cuda  = torch.from_numpy(graph_edges_weights).cuda().unsqueeze(0)
        graph_clusters_cuda       = torch.from_numpy(graph_clusters).cuda().unsqueeze(0)
        pixel_anchors_cuda        = torch.from_numpy(pixel_anchors).cuda().unsqueeze(0)
        pixel_weights_cuda        = torch.from_numpy(pixel_weights).cuda().unsqueeze(0)
        intrinsics_cuda           = torch.from_numpy(intrinsics).cuda().unsqueeze(0)

        num_nodes_cuda            = torch.from_numpy(num_nodes).cuda().unsqueeze(0)

        with torch.no_grad():
            model_data = self.model(
                source_cuda, target_cuda, 
                graph_nodes_cuda, graph_edges_cuda, graph_edges_weights_cuda, graph_clusters_cuda, 
                pixel_anchors_cuda, pixel_weights_cuda, 
                num_nodes_cuda, intrinsics_cuda, 
                evaluate=True, split="test"
            )

        # Get some of the results
        rotations_pred    = model_data["node_rotations"].view(num_nodes, 3, 3).cpu().numpy()
        translations_pred = model_data["node_translations"].view(num_nodes, 3).cpu().numpy()

        mask_pred = model_data["mask_pred"]
        assert mask_pred is not None, "Make sure use_mask=True in options.py"
        mask_pred = mask_pred.view(-1, opt.image_height, opt.image_width).cpu().numpy()

        # Compute mask gt for mask baseline
        _, source_points, valid_source_points, target_matches, \
            valid_target_matches, valid_correspondences, _, \
                _ = model_data["correspondence_info"]
        print(valid_correspondences.shape)
        print(valid_correspondences)
        target_matches        = target_matches.view(-1, opt.image_height, opt.image_width).cpu().numpy()
        valid_source_points   = valid_source_points.view(-1, opt.image_height, opt.image_width).cpu().numpy()
        valid_target_matches  = valid_target_matches.view(-1, opt.image_height, opt.image_width).cpu().numpy()
        valid_correspondences = valid_correspondences.view(-1, opt.image_height, opt.image_width).cpu().numpy()

        deformed_graph_nodes = graph_nodes + translations_pred        
        
        del source_cuda
        del target_cuda
        del target_boundary_mask_cuda
        del graph_nodes_cuda
        del graph_edges_cuda
        del graph_edges_weights_cuda
        del graph_clusters_cuda
        del pixel_anchors_cuda
        del pixel_weights_cuda
        del intrinsics_cuda
        
        source_flat = np.moveaxis(source, 0, -1).reshape(-1, 6)
        source_points = viz_utils.transform_pointcloud_to_opengl_coords(source_flat[..., 3:])
        source_colors = source_flat[..., :3]

        source_pcd = o3d.geometry.PointCloud()
        source_pcd.points = o3d.utility.Vector3dVector(source_points)
        source_pcd.colors = o3d.utility.Vector3dVector(source_colors)

        # keep only object using the mask
        valid_source_mask = np.moveaxis(valid_source_points, 0, -1).reshape(-1).astype(np.bool)
        valid_source_points = source_points[valid_source_mask, :]
        valid_source_colors = source_colors[valid_source_mask, :]
        # source object PointCloud
        source_object_pcd = o3d.geometry.PointCloud()
        source_object_pcd.points = o3d.utility.Vector3dVector(valid_source_points)
        source_object_pcd.colors = o3d.utility.Vector3dVector(valid_source_colors)

        # o3d.visualization.draw_geometries([source_pcd])
        # o3d.visualization.draw_geometries([source_object_pcd])

        #####################################################################################################
        # Source warped
        #####################################################################################################
        warped_deform_pred_3d_np = image_proc.warp_deform_3d(
            source, pixel_anchors, pixel_weights, graph_nodes, rotations_pred, translations_pred
        )

        source_warped = np.copy(source)
        source_warped[3:, :, :] = warped_deform_pred_3d_np

        # (source) warped RGB-D image
        source_warped = np.moveaxis(source_warped, 0, -1).reshape(-1, 6)
        warped_points = viz_utils.transform_pointcloud_to_opengl_coords(source_warped[..., 3:])
        warped_colors = source_warped[..., :3]
        # Filter points at (0, 0, 0)
        warped_points = warped_points[valid_source_mask]
        warped_colors = warped_colors[valid_source_mask]
        # warped PointCloud
        warped_pcd = o3d.geometry.PointCloud()
        warped_pcd.points = o3d.utility.Vector3dVector(warped_points)
        warped_pcd.paint_uniform_color([1, 0.706, 0]) # warped_pcd.colors = o3d.utility.Vector3dVector(warped_colors)

        # o3d.visualization.draw_geometries([source_object_pcd, warped_pcd])

        ####################################
        # TARGET #
        ####################################
        # target RGB-D image
        target_flat = np.moveaxis(target, 0, -1).reshape(-1, 6)
        target_points = viz_utils.transform_pointcloud_to_opengl_coords(target_flat[..., 3:])
        target_colors = target_flat[..., :3]
        # target PointCloud
        target_pcd = o3d.geometry.PointCloud()
        target_pcd.points = o3d.utility.Vector3dVector(target_points)
        target_pcd.colors = o3d.utility.Vector3dVector(target_colors)

        # o3d.visualization.draw_geometries([target_pcd])

        ####################################
        # GRAPH #
        ####################################

        # Transform to OpenGL coords
        graph_nodes = viz_utils.transform_pointcloud_to_opengl_coords(graph_nodes)
        deformed_graph_nodes = viz_utils.transform_pointcloud_to_opengl_coords(deformed_graph_nodes)

        # Graph nodes
        rendered_graph_nodes = []
        for node in graph_nodes:
            mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
            mesh_sphere.compute_vertex_normals()
            mesh_sphere.paint_uniform_color([1.0, 0.0, 0.0])
            mesh_sphere.translate(node)
            rendered_graph_nodes.append(mesh_sphere)

        # Merge all different sphere meshes
        rendered_graph_nodes = viz_utils.merge_meshes(rendered_graph_nodes)

        # Graph edges
        edges_pairs = []
        for node_id, edges in enumerate(graph_edges):
            for neighbor_id in edges:
                if neighbor_id == -1:
                    break
                edges_pairs.append([node_id, neighbor_id])    

        colors = [[0.2, 1.0, 0.2] for i in range(len(edges_pairs))]
        line_mesh = line_mesh_utils.LineMesh(graph_nodes, edges_pairs, colors, radius=0.003)
        line_mesh_geoms = line_mesh.cylinder_segments

        # Merge all different line meshes
        line_mesh_geoms = viz_utils.merge_meshes(line_mesh_geoms)

        # o3d.visualization.draw_geometries([rendered_graph_nodes, line_mesh_geoms, source_object_pcd])

        # Combined nodes & edges
        rendered_graph = [rendered_graph_nodes, line_mesh_geoms]

        ####################################
        # Mask
        ####################################
        mask_pred_flat = mask_pred.reshape(-1)
        valid_correspondences = valid_correspondences.reshape(-1).astype(np.bool)

        ####################################
        # Correspondences
        ####################################
        # target matches
        target_matches = np.moveaxis(target_matches, 0, -1).reshape(-1, 3)
        target_matches = viz_utils.transform_pointcloud_to_opengl_coords(target_matches)

        ################################
        # "Good" matches
        ################################
        good_mask = valid_correspondences & (mask_pred_flat >= weight_thr)
        good_source_points_corresp  = source_points[good_mask]
        good_target_matches_corresp = target_matches[good_mask]
        good_mask_pred              = mask_pred_flat[good_mask]

        # number of good matches
        n_good_matches = good_source_points_corresp.shape[0]
        # Subsample
        subsample = True
        if subsample:
            N = 2000
            sampled_idxs = np.random.permutation(n_good_matches)[:N]
            good_source_points_corresp  = good_source_points_corresp[sampled_idxs]
            good_target_matches_corresp = good_target_matches_corresp[sampled_idxs]
            good_mask_pred              = good_mask_pred[sampled_idxs]
            n_good_matches = N
        # both good_source and good_target points together into one vector
        good_matches_points = np.concatenate([good_source_points_corresp, good_target_matches_corresp], axis=0)
        good_matches_lines = [[i, i + n_good_matches] for i in range(0, n_good_matches, 1)]

        # --> Create good (unweighted) lines 
        good_matches_colors = [[201/255, 177/255, 14/255] for i in range(len(good_matches_lines))]
        good_matches_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(good_matches_points),
            lines=o3d.utility.Vector2iVector(good_matches_lines),
        )
        good_matches_set.colors = o3d.utility.Vector3dVector(good_matches_colors)

        # --> Create good weighted lines 
        # first, we need to get the proper color coding
        high_color, low_color = np.array([0.0, 0.8, 0]), np.array([0.8, 0, 0.0])

        good_weighted_matches_colors = np.ones_like(good_source_points_corresp)

        weights_normalized = np.maximum(np.minimum(0.5 + (good_mask_pred - weight_thr) / weight_scale, 1.0), 0.0)
        weights_normalized_opposite = 1 - weights_normalized

        good_weighted_matches_colors[:, 0] = weights_normalized * high_color[0] + weights_normalized_opposite * low_color[0]
        good_weighted_matches_colors[:, 1] = weights_normalized * high_color[1] + weights_normalized_opposite * low_color[1]
        good_weighted_matches_colors[:, 2] = weights_normalized * high_color[2] + weights_normalized_opposite * low_color[2]

        good_weighted_matches_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(good_matches_points),
            lines=o3d.utility.Vector2iVector(good_matches_lines),
        )
        good_weighted_matches_set.colors = o3d.utility.Vector3dVector(good_weighted_matches_colors)


        ################################
        # "Bad" matches
        ################################
        bad_mask = valid_correspondences & (mask_pred_flat < weight_thr)
        bad_source_points_corresp  = source_points[bad_mask]
        bad_target_matches_corresp = target_matches[bad_mask]
        bad_mask_pred              = mask_pred_flat[bad_mask]

        # number of good matches
        n_bad_matches = bad_source_points_corresp.shape[0]

        # both good_source and good_target points together into one vector
        bad_matches_points = np.concatenate([bad_source_points_corresp, bad_target_matches_corresp], axis=0)
        bad_matches_lines = [[i, i + n_bad_matches] for i in range(0, n_bad_matches, 1)]

        # --> Create bad (unweighted) lines 
        bad_matches_colors = [[201/255, 177/255, 14/255] for i in range(len(bad_matches_lines))]
        bad_matches_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(bad_matches_points),
            lines=o3d.utility.Vector2iVector(bad_matches_lines),
        )
        bad_matches_set.colors = o3d.utility.Vector3dVector(bad_matches_colors)

        # --> Create bad weighted lines 
        # first, we need to get the proper color coding
        high_color, low_color = np.array([0.0, 0.8, 0]), np.array([0.8, 0, 0.0])

        bad_weighted_matches_colors = np.ones_like(bad_source_points_corresp)

        weights_normalized = np.maximum(np.minimum(0.5 + (bad_mask_pred - weight_thr) / weight_scale, 1.0), 0.0)
        weights_normalized_opposite = 1 - weights_normalized

        bad_weighted_matches_colors[:, 0] = weights_normalized * high_color[0] + weights_normalized_opposite * low_color[0]
        bad_weighted_matches_colors[:, 1] = weights_normalized * high_color[1] + weights_normalized_opposite * low_color[1]
        bad_weighted_matches_colors[:, 2] = weights_normalized * high_color[2] + weights_normalized_opposite * low_color[2]

        bad_weighted_matches_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(bad_matches_points),
            lines=o3d.utility.Vector2iVector(bad_matches_lines),
        )
        bad_weighted_matches_set.colors = o3d.utility.Vector3dVector(bad_weighted_matches_colors)

        ####################################
        # Generate info for aligning source to target (by interpolating between source and warped source)
        ####################################
        assert warped_points.shape[0] == valid_source_points.shape[0]
        line_segments = warped_points - valid_source_points
        line_segments_unit, line_lengths = line_mesh_utils.normalized(line_segments)
        line_lengths = line_lengths[:, np.newaxis]
        line_lengths = np.repeat(line_lengths, 3, axis=1)

        ####################################
        # Draw 
        ####################################
        geometry_dict = {
            "source_pcd": source_pcd, 
            "source_obj": source_object_pcd, 
            "target_pcd": target_pcd, 
            "graph":      rendered_graph,
            # "deformed_graph":    rendered_deformed_graph
        }

        alignment_dict = {
            "valid_source_points": valid_source_points,
            "line_segments_unit":  line_segments_unit,
            "line_lengths":        line_lengths
        }

        matches_dict = {
            "good_matches_set":          good_matches_set,
            "good_weighted_matches_set": good_weighted_matches_set,
            "bad_matches_set":           bad_matches_set,
            "bad_weighted_matches_set":  bad_weighted_matches_set
        }

        # Save image 
        sourceimage = create_image([source_object_pcd,good_weighted_matches_set])
        warpedimage = create_image([warped_pcd])
        targetimage = create_image([target_pcd],w=1280,h=960)
        image = cv2.hconcat([cv2.vconcat([sourceimage,warpedimage]),targetimage])
        
        self.image_list.append(image)
        #####################################################################################################
        # Open viewer
        #####################################################################################################
#         manager = viz_utils.CustomDrawGeometryWithKeyCallback(
#             geometry_dict, alignment_dict, matches_dict
#         )
#         manager.custom_draw_geometry_with_key_callback()
        

In [6]:
def create_image(point_cloud_list,w=640,h=480,sideview=False):
	vis = o3d.visualization.Visualizer()
	vis.create_window(width=w, height=h)
	# Read camera params
	# param = o3d.io.read_pinhole_camera_parameters('cameraparams.json')
	param = o3d.io.read_pinhole_camera_parameters('./viewpoint.json' if w == 640 else './viewpoint_big.json')
	ctr = vis.get_view_control()
	ctr.convert_from_pinhole_camera_parameters(param)
	for pc in point_cloud_list:
		vis.add_geometry(pc)


	if sideview: 
		ctr.rotate(480,0) # Front view
	else:
		ctr.rotate(0, 0) # Side view

	vis.poll_events()
	vis.update_renderer()
	image = vis.capture_screen_float_buffer()
	image = np.asarray(image) * 255
	image = image.astype(np.uint8)

# 	vis.run()
	# Close
	vis.destroy_window()
	return image

In [7]:
example_selector = ExampleSelector(jsonfile="../DeepDeform/train_graphs.json")


Dropdown(description='Example:', options=('seq000-blackdog-000000-000200', 'seq000-blackdog-000000-001200', 's…

1/100 Completed
train/seq028/color/000051.jpg train/seq028/depth/000051.png
Graph Nodes (208, 3)
[[-0.20947239  0.2787412   0.704     ]
 [-0.09223369  0.2791858   0.696     ]
 [ 0.0336504   0.27676147  0.699     ]
 [-0.3285019  -0.40425992  1.21      ]
 [-0.24472183 -0.39865717  1.181     ]
 [-0.28884578  0.27758133  0.692     ]
 [-0.14670879  0.2693017   0.708     ]
 [-0.33590087  0.26173824  0.685     ]
 [-0.38577354 -0.405033    1.225     ]
 [-0.16728395 -0.41197646  1.246     ]
 [-0.01864186  0.26459068  0.702     ]
 [-0.25210458  0.25803173  0.721     ]
 [-0.3054478  -0.37875167  1.17      ]
 [-0.11109734 -0.4036781   1.247     ]
 [-0.56634045 -0.4002329   1.243     ]
 [-0.49292046 -0.41665438  1.294     ]
 [-0.4511888  -0.3950491   1.247     ]
 [-0.26311913 -0.3686779   1.145     ]
 [ 0.00988799  0.24113506  0.738     ]
 [-0.35628283  0.2340557   0.724     ]
 [-0.17842938  0.22747952  0.727     ]
 [-0.04016414  0.23762646  0.739     ]
 [-0.30301753  0.23179272  0.717     ]
 [-0.1

torch.Size([1, 448, 640])
tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]], device='cuda:0')


In [4]:
def read_depth_image(depth_path,thresh=0):
	depth_image = np.array(plt.imread(depth_path))

	f_x = 525
	f_y = 525
	c_x = 319.5
	c_y = 239.5

	y,x = np.where(depth_image > thresh)
	z = depth_image[y,x]
	point_cloud = np.array([ z*(x - c_x)/f_x,  z*(y - c_y)/f_y, z]).T
	point2frame = np.array([[y[i],x[i]] for i in range(point_cloud.shape[0]) ])    
	return point_cloud,point2frame


In [32]:
class DeformableModel():
	def __init__(self,sourcepath,grid_size,k=8,use3D=False):
		super(DeformableModel,self).__init__()
		"""
			Here we define our deformable model
			@param => datapath => Path to the source depth scan/mesh/point cloud :: file path::
			@param => grid_size => Voxel grid size to perform uniform sampling for creating deformable model  :: float 
			@param => use3D => # Connectivity is better defined as the image instead of the dicontinouty present in depth scan :: bool
		"""
		
		self.k = k # Copy data for later use
		self.use3D = use3D

		self.vertices,self.point2frame = read_depth_image(sourcepath)
		self.nodes = self.get_nodes_from_pointcloud(grid_size)
		self.N = self.nodes.shape[0]	
		self.V = self.vertices.shape[0]	

		self.create_adj_list(k=k) # Create edges and mapping for graph
		print(f"Deformable Model: Initialization Complete, Node:{self.nodes.shape[0]} Edges:{self.adj_matrix[0].shape[0]//2}")


	def get_nodes_from_pointcloud(self,grid_size):
		"""
			Create the deformable model by uniform sampling and choosing k points 
			# Refernce: https://github.com/pglira/Point_cloud_tools_for_Matlab/blob/master/classes/4pointCloud/uniformSampling.m ; 
			@param => grid_size => Voxel grid size to perform uniform sampling for creating deformable model  :: float 
			return => node :: Graph nodes :: Nx3 np.array     
		"""
		vertices = self.vertices
		V = vertices.shape[0]

		minPoi = np.min(vertices,axis=0,keepdims=True) # Point with smallest coordinates
		localOrigin = np.floor(minPoi/100)*100 # Rounded local origin for voxel structure (voxels of different pcs have coincident voxel centers if mod(100, voxelSize) == 0)
		
		# Find 2/3-dimensional indices of voxels in which points are lying
		idxVoxelUnique = np.floor((vertices - localOrigin)/grid_size)
		# Remove multiple voxels
		idxVoxelUnique, indUnique = np.unique(idxVoxelUnique,axis=0,return_inverse=True)	# indUnique contains "voxel index" for each point	

		VoxelCenter = localOrigin + grid_size/2 + idxVoxelUnique*grid_size

		N = idxVoxelUnique.shape[0] # Number of unique points
		# Select points nearest to voxel centers ---------------------------------------
		idxSort = np.argsort(indUnique)
		indUnique = indUnique[idxSort]
		vertices = vertices[idxSort]
		idxJump = np.where(np.diff(indUnique))[0] # Sort indices and points (in order to find points inside of voxels very fast in the next loop)
		ind_list = []
		for i in range(N):
    		# Distance of points to voxel center
			if i == 0:
				dist2voxelCenter = np.linalg.norm(vertices[:idxJump[0]+1] - VoxelCenter[i],axis=1)
				selected_ind = np.argmin(dist2voxelCenter)
			elif i == N-1:
				dist2voxelCenter = np.linalg.norm(vertices[idxJump[i-1]+1:] - VoxelCenter[i],axis=1)
				selected_ind = np.argmin(dist2voxelCenter) + idxJump[i-1] +1
			else:
				dist2voxelCenter = np.linalg.norm(vertices[idxJump[i-1]+1:idxJump[i]+1] - VoxelCenter[i],axis=1)
				selected_ind = np.argmin(dist2voxelCenter) + idxJump[i-1] + 1 

			selected_ind = idxSort[selected_ind]	
			ind_list.append(selected_ind)

		self.node2vert = sorted(ind_list)
		return self.vertices[self.node2vert]

	def create_adj_list(self,k=4):
		"""
			Define the neighbourhood, vertex-node mapping & corresponding weight-matrix as described in the paper 	
			@param => k:: int :: number of closest points on the graph for every vertex as defined in the paper
		"""

		if self.use3D == True:
			kdtree = KDTree(self.nodes) # Use KD-tree for faster computation
			_,closest_neighbours = kdtree.query(self.vertices,k=k+1) # Find the k+1 clostest nodes for every vertex on the source points
		else:	
			kdtree = KDTree(self.nodes[:,:2]) # Use KD-tree for faster computation
			_,closest_neighbours = kdtree.query(self.vertices[:,:2],k=k+1) # Find the k+1 clostest nodes for every vertex on the source points

		self.v2n = closest_neighbours[:,:-1] # Define the vertex to node mapping  (closest k)

		N = self.nodes.shape[0]
		self.adj_matrix = np.zeros((N,N),dtype=np.int32) # Define adj matrix
		
		st = time.time()

		# For every vertex connect all distinct k vertexes
		for nn in np.unique(closest_neighbours,axis=1):
			inds = np.unique(nn[:-1])
			if len(inds) > 1:
				for i,x in enumerate(inds):
					for j,y in enumerate(inds[i+1:]):
						self.adj_matrix[x,y] = self.adj_matrix[y,x] = 1

		# Calculate the weight of influence from every node to vertex
		if self.use3D:
			self.w2n = np.linalg.norm(self.vertices[:,None,:] - self.nodes[closest_neighbours,:],axis=2)
		else:
			self.w2n = np.linalg.norm(self.vertices[:,None,:2] - self.nodes[closest_neighbours,:2],axis=2)

		self.w2n = 1 - self.w2n[:,:k]/self.w2n[:,k:k+1]
		self.w2n = self.w2n/np.sum(self.w2n,axis=1,keepdims=True)
    
	def generate(self):
		edges = np.where(self.adj_matrix==1)        
		dmodel["graph_nodes"] = self.nodes
		N = self.nodes[0]        
		dmodel["graph_edges"] = -1*np.ones((self.nodes,self.k))
		dmodel["graph_edge_weights"] = np.zeros((self.nodes,self.k))
		for i in range(N):
			cnt = 0     
			for j in range(N):
				if self.adj_matrix[i,j] == 1:
				dmodel["graph_edges"][i,cnt] = j
					cnt += 1
        

SyntaxError: invalid syntax (<ipython-input-32-046772715ca9>, line 108)

In [24]:
class RGBDVideoResults():
    def __init__(self,datapath="",source_id=0,gridsize=0.003):

        saved_model = opt.saved_model

        assert os.path.isfile(saved_model), f"Model {saved_model} does not exist."
        pretrained_dict = torch.load(saved_model)

        # Construct model
        self.model = DeformNet().cuda()

        if "chairs_things" in saved_model:
            self.model.flow_net.load_state_dict(pretrained_dict)
        else:
            if opt.model_module_to_load == "full_model":
                # Load completely model            
                self.model.load_state_dict(pretrained_dict)
            elif opt.model_module_to_load == "only_flow_net":
                # Load only optical flow part
                self.model_dict = model.state_dict()
                # 1. filter out unnecessary keys
                pretrained_dict = {k: v for k, v in pretrained_dict.items() if "flow_net" in k}
                # 2. overwrite entries in the existing state dict
                self.model_dict.update(pretrained_dict) 
                # 3. load the new state dict
                self.model.load_state_dict(model_dict)
            else:
                print(opt.model_module_to_load, "is not a valid argument (A: 'full_model', B: 'only_flow_net')")
                exit()

        self.model.eval()
        self.datapath = datapath 
        
        filelist = sorted(os.listdir(os.path.join(datapath,"depth")),key=lambda x: int(x.split('.')[0]))
        sourcefile = filelist[source_id]
        dmodel = DeformableModel(os.path.join(datapath,"depth",sourcefile),gridsize).generate()
        for targetfile in filelist:
            print(f"{i}/{len(filelist)} Completed")
            try: 
                self.load_example(sourcefile,targetfile,dmodel)
            except Exception as e:
                print("Error:",e)
                continue
        imageio.mimsave('movie.gif', self.image_list)
    def load_image(self,sourcepath,targetpath,dmodel):
        
        src_color_image_path         = os.path.join(datapath,"color",sourcepath)
        src_depth_image_path         = os.path.join(datapath,"depth",sourcepath)
        tgt_color_image_path         = os.path.join(datapath,"color",targetpath)
        tgt_depth_image_path         = os.path.join(datapath,"color",sourcepath)

        # Make sure to use the intrinsics corresponding to the seq_id above!!  
        intrinsics = {
            "fx": 525.0,
            "fy": 525.0,
            "cx": 319.5,
            "cy": 239.5
        }

        # Some params for coloring the predicted correspondence confidences
        weight_thr = 0.3
        weight_scale = 1


        image_height = opt.image_height
        image_width  = opt.image_width
        max_boundary_dist = opt.max_boundary_dist

        
        
        # Source color and depth
        source, _, cropper = dataset.DeformDataset.load_image(
            src_color_image_path, src_depth_image_path, intrinsics, image_height, image_width
        )

        # Target color and depth (and boundary mask)
        target, target_boundary_mask, _ = dataset.DeformDataset.load_image(
            tgt_color_image_path, tgt_depth_image_path, intrinsics, image_height, image_width, cropper=cropper,
            max_boundary_dist=max_boundary_dist, compute_boundary_mask=True
        )

        # Graph
        graph_nodes = dmodel['graph_nodes']
        graph_edges = dmodel['graph_edges']
        graph_edges_weights = dmodel['graph_edges_weights']
        graph_clusters = dmodel['graph_clusters']
        pixel_anchors  = dmodel['pixel_anchors']
        pixel_weights = dmodel['pixel_weights']
        
        num_nodes = np.array(graph_nodes.shape[0], dtype=np.int64)
                
        # Update intrinsics to reflect the crops
        fx, fy, cx, cy = image_proc.modify_intrinsics_due_to_cropping(
            intrinsics['fx'], intrinsics['fy'], intrinsics['cx'], intrinsics['cy'], 
            image_height, image_width, original_h=cropper.h, original_w=cropper.w
        )

        intrinsics = np.zeros((4), dtype=np.float32)
        intrinsics[0] = fx
        intrinsics[1] = fy
        intrinsics[2] = cx
        intrinsics[3] = cy
        

        #####################################################################################################
        # Predict deformation
        #####################################################################################################

        # Move to device and unsqueeze in the batch dimension (to have batch size 1)
        source_cuda               = torch.from_numpy(source).cuda().unsqueeze(0)
        target_cuda               = torch.from_numpy(target).cuda().unsqueeze(0)
        target_boundary_mask_cuda = torch.from_numpy(target_boundary_mask).cuda().unsqueeze(0)
        graph_nodes_cuda          = torch.from_numpy(graph_nodes).cuda().unsqueeze(0)
        graph_edges_cuda          = torch.from_numpy(graph_edges).cuda().unsqueeze(0)
        graph_edges_weights_cuda  = torch.from_numpy(graph_edges_weights).cuda().unsqueeze(0)
        graph_clusters_cuda       = torch.from_numpy(graph_clusters).cuda().unsqueeze(0)
        pixel_anchors_cuda        = torch.from_numpy(pixel_anchors).cuda().unsqueeze(0)
        pixel_weights_cuda        = torch.from_numpy(pixel_weights).cuda().unsqueeze(0)
        intrinsics_cuda           = torch.from_numpy(intrinsics).cuda().unsqueeze(0)

        num_nodes_cuda            = torch.from_numpy(num_nodes).cuda().unsqueeze(0)

        with torch.no_grad():
            model_data = self.model(
                source_cuda, target_cuda, 
                graph_nodes_cuda, graph_edges_cuda, graph_edges_weights_cuda, graph_clusters_cuda, 
                pixel_anchors_cuda, pixel_weights_cuda, 
                num_nodes_cuda, intrinsics_cuda, 
                evaluate=True, split="test"
            )

        # Get some of the results
        rotations_pred    = model_data["node_rotations"].view(num_nodes, 3, 3).cpu().numpy()
        translations_pred = model_data["node_translations"].view(num_nodes, 3).cpu().numpy()

        mask_pred = model_data["mask_pred"]
        assert mask_pred is not None, "Make sure use_mask=True in options.py"
        mask_pred = mask_pred.view(-1, opt.image_height, opt.image_width).cpu().numpy()

        # Compute mask gt for mask baseline
        _, source_points, valid_source_points, target_matches, \
            valid_target_matches, valid_correspondences, _, \
                _ = model_data["correspondence_info"]

        target_matches        = target_matches.view(-1, opt.image_height, opt.image_width).cpu().numpy()
        valid_source_points   = valid_source_points.view(-1, opt.image_height, opt.image_width).cpu().numpy()
        valid_target_matches  = valid_target_matches.view(-1, opt.image_height, opt.image_width).cpu().numpy()
        valid_correspondences = valid_correspondences.view(-1, opt.image_height, opt.image_width).cpu().numpy()

        deformed_graph_nodes = graph_nodes + translations_pred        
        
        del source_cuda
        del target_cuda
        del target_boundary_mask_cuda
        del graph_nodes_cuda
        del graph_edges_cuda
        del graph_edges_weights_cuda
        del graph_clusters_cuda
        del pixel_anchors_cuda
        del pixel_weights_cuda
        del intrinsics_cuda
        
        source_flat = np.moveaxis(source, 0, -1).reshape(-1, 6)
        source_points = viz_utils.transform_pointcloud_to_opengl_coords(source_flat[..., 3:])
        source_colors = source_flat[..., :3]

        source_pcd = o3d.geometry.PointCloud()
        source_pcd.points = o3d.utility.Vector3dVector(source_points)
        source_pcd.colors = o3d.utility.Vector3dVector(source_colors)

        # keep only object using the mask
        valid_source_mask = np.moveaxis(valid_source_points, 0, -1).reshape(-1).astype(np.bool)
        valid_source_points = source_points[valid_source_mask, :]
        valid_source_colors = source_colors[valid_source_mask, :]
        # source object PointCloud
        source_object_pcd = o3d.geometry.PointCloud()
        source_object_pcd.points = o3d.utility.Vector3dVector(valid_source_points)
        source_object_pcd.colors = o3d.utility.Vector3dVector(valid_source_colors)

        # o3d.visualization.draw_geometries([source_pcd])
        # o3d.visualization.draw_geometries([source_object_pcd])

        #####################################################################################################
        # Source warped
        #####################################################################################################
        warped_deform_pred_3d_np = image_proc.warp_deform_3d(
            source, pixel_anchors, pixel_weights, graph_nodes, rotations_pred, translations_pred
        )

        source_warped = np.copy(source)
        source_warped[3:, :, :] = warped_deform_pred_3d_np

        # (source) warped RGB-D image
        source_warped = np.moveaxis(source_warped, 0, -1).reshape(-1, 6)
        warped_points = viz_utils.transform_pointcloud_to_opengl_coords(source_warped[..., 3:])
        warped_colors = source_warped[..., :3]
        # Filter points at (0, 0, 0)
        warped_points = warped_points[valid_source_mask]
        warped_colors = warped_colors[valid_source_mask]
        # warped PointCloud
        warped_pcd = o3d.geometry.PointCloud()
        warped_pcd.points = o3d.utility.Vector3dVector(warped_points)
        warped_pcd.paint_uniform_color([1, 0.706, 0]) # warped_pcd.colors = o3d.utility.Vector3dVector(warped_colors)

        # o3d.visualization.draw_geometries([source_object_pcd, warped_pcd])

        ####################################
        # TARGET #
        ####################################
        # target RGB-D image
        target_flat = np.moveaxis(target, 0, -1).reshape(-1, 6)
        target_points = viz_utils.transform_pointcloud_to_opengl_coords(target_flat[..., 3:])
        target_colors = target_flat[..., :3]
        # target PointCloud
        target_pcd = o3d.geometry.PointCloud()
        target_pcd.points = o3d.utility.Vector3dVector(target_points)
        target_pcd.colors = o3d.utility.Vector3dVector(target_colors)

        # o3d.visualization.draw_geometries([target_pcd])

        ####################################
        # GRAPH #
        ####################################

        # Transform to OpenGL coords
        graph_nodes = viz_utils.transform_pointcloud_to_opengl_coords(graph_nodes)
        deformed_graph_nodes = viz_utils.transform_pointcloud_to_opengl_coords(deformed_graph_nodes)

        # Graph nodes
        rendered_graph_nodes = []
        for node in graph_nodes:
            mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
            mesh_sphere.compute_vertex_normals()
            mesh_sphere.paint_uniform_color([1.0, 0.0, 0.0])
            mesh_sphere.translate(node)
            rendered_graph_nodes.append(mesh_sphere)

        # Merge all different sphere meshes
        rendered_graph_nodes = viz_utils.merge_meshes(rendered_graph_nodes)

        # Graph edges
        edges_pairs = []
        for node_id, edges in enumerate(graph_edges):
            for neighbor_id in edges:
                if neighbor_id == -1:
                    break
                edges_pairs.append([node_id, neighbor_id])    

        colors = [[0.2, 1.0, 0.2] for i in range(len(edges_pairs))]
        line_mesh = line_mesh_utils.LineMesh(graph_nodes, edges_pairs, colors, radius=0.003)
        line_mesh_geoms = line_mesh.cylinder_segments

        # Merge all different line meshes
        line_mesh_geoms = viz_utils.merge_meshes(line_mesh_geoms)

        # o3d.visualization.draw_geometries([rendered_graph_nodes, line_mesh_geoms, source_object_pcd])

        # Combined nodes & edges
        rendered_graph = [rendered_graph_nodes, line_mesh_geoms]

        ####################################
        # Mask
        ####################################
        mask_pred_flat = mask_pred.reshape(-1)
        valid_correspondences = valid_correspondences.reshape(-1).astype(np.bool)

        ####################################
        # Correspondences
        ####################################
        # target matches
        target_matches = np.moveaxis(target_matches, 0, -1).reshape(-1, 3)
        target_matches = viz_utils.transform_pointcloud_to_opengl_coords(target_matches)

        ################################
        # "Good" matches
        ################################
        good_mask = valid_correspondences & (mask_pred_flat >= weight_thr)
        good_source_points_corresp  = source_points[good_mask]
        good_target_matches_corresp = target_matches[good_mask]
        good_mask_pred              = mask_pred_flat[good_mask]

        # number of good matches
        n_good_matches = good_source_points_corresp.shape[0]
        # Subsample
        subsample = True
        if subsample:
            N = 2000
            sampled_idxs = np.random.permutation(n_good_matches)[:N]
            good_source_points_corresp  = good_source_points_corresp[sampled_idxs]
            good_target_matches_corresp = good_target_matches_corresp[sampled_idxs]
            good_mask_pred              = good_mask_pred[sampled_idxs]
            n_good_matches = N
        # both good_source and good_target points together into one vector
        good_matches_points = np.concatenate([good_source_points_corresp, good_target_matches_corresp], axis=0)
        good_matches_lines = [[i, i + n_good_matches] for i in range(0, n_good_matches, 1)]

        # --> Create good (unweighted) lines 
        good_matches_colors = [[201/255, 177/255, 14/255] for i in range(len(good_matches_lines))]
        good_matches_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(good_matches_points),
            lines=o3d.utility.Vector2iVector(good_matches_lines),
        )
        good_matches_set.colors = o3d.utility.Vector3dVector(good_matches_colors)

        # --> Create good weighted lines 
        # first, we need to get the proper color coding
        high_color, low_color = np.array([0.0, 0.8, 0]), np.array([0.8, 0, 0.0])

        good_weighted_matches_colors = np.ones_like(good_source_points_corresp)

        weights_normalized = np.maximum(np.minimum(0.5 + (good_mask_pred - weight_thr) / weight_scale, 1.0), 0.0)
        weights_normalized_opposite = 1 - weights_normalized

        good_weighted_matches_colors[:, 0] = weights_normalized * high_color[0] + weights_normalized_opposite * low_color[0]
        good_weighted_matches_colors[:, 1] = weights_normalized * high_color[1] + weights_normalized_opposite * low_color[1]
        good_weighted_matches_colors[:, 2] = weights_normalized * high_color[2] + weights_normalized_opposite * low_color[2]

        good_weighted_matches_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(good_matches_points),
            lines=o3d.utility.Vector2iVector(good_matches_lines),
        )
        good_weighted_matches_set.colors = o3d.utility.Vector3dVector(good_weighted_matches_colors)


        ################################
        # "Bad" matches
        ################################
        bad_mask = valid_correspondences & (mask_pred_flat < weight_thr)
        bad_source_points_corresp  = source_points[bad_mask]
        bad_target_matches_corresp = target_matches[bad_mask]
        bad_mask_pred              = mask_pred_flat[bad_mask]

        # number of good matches
        n_bad_matches = bad_source_points_corresp.shape[0]

        # both good_source and good_target points together into one vector
        bad_matches_points = np.concatenate([bad_source_points_corresp, bad_target_matches_corresp], axis=0)
        bad_matches_lines = [[i, i + n_bad_matches] for i in range(0, n_bad_matches, 1)]

        # --> Create bad (unweighted) lines 
        bad_matches_colors = [[201/255, 177/255, 14/255] for i in range(len(bad_matches_lines))]
        bad_matches_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(bad_matches_points),
            lines=o3d.utility.Vector2iVector(bad_matches_lines),
        )
        bad_matches_set.colors = o3d.utility.Vector3dVector(bad_matches_colors)

        # --> Create bad weighted lines 
        # first, we need to get the proper color coding
        high_color, low_color = np.array([0.0, 0.8, 0]), np.array([0.8, 0, 0.0])

        bad_weighted_matches_colors = np.ones_like(bad_source_points_corresp)

        weights_normalized = np.maximum(np.minimum(0.5 + (bad_mask_pred - weight_thr) / weight_scale, 1.0), 0.0)
        weights_normalized_opposite = 1 - weights_normalized

        bad_weighted_matches_colors[:, 0] = weights_normalized * high_color[0] + weights_normalized_opposite * low_color[0]
        bad_weighted_matches_colors[:, 1] = weights_normalized * high_color[1] + weights_normalized_opposite * low_color[1]
        bad_weighted_matches_colors[:, 2] = weights_normalized * high_color[2] + weights_normalized_opposite * low_color[2]

        bad_weighted_matches_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(bad_matches_points),
            lines=o3d.utility.Vector2iVector(bad_matches_lines),
        )
        bad_weighted_matches_set.colors = o3d.utility.Vector3dVector(bad_weighted_matches_colors)

        ####################################
        # Generate info for aligning source to target (by interpolating between source and warped source)
        ####################################
        assert warped_points.shape[0] == valid_source_points.shape[0]
        line_segments = warped_points - valid_source_points
        line_segments_unit, line_lengths = line_mesh_utils.normalized(line_segments)
        line_lengths = line_lengths[:, np.newaxis]
        line_lengths = np.repeat(line_lengths, 3, axis=1)

        ####################################
        # Draw 
        ####################################
        geometry_dict = {
            "source_pcd": source_pcd, 
            "source_obj": source_object_pcd, 
            "target_pcd": target_pcd, 
            "graph":      rendered_graph,
            # "deformed_graph":    rendered_deformed_graph
        }

        alignment_dict = {
            "valid_source_points": valid_source_points,
            "line_segments_unit":  line_segments_unit,
            "line_lengths":        line_lengths
        }

        matches_dict = {
            "good_matches_set":          good_matches_set,
            "good_weighted_matches_set": good_weighted_matches_set,
            "bad_matches_set":           bad_matches_set,
            "bad_weighted_matches_set":  bad_weighted_matches_set
        }

        # Save image 
        sourceimage = create_image([source_object_pcd,good_weighted_matches_set])
        warpedimage = create_image([warped_pcd])
        targetimage = create_image([target_pcd],w=1280,h=960)
        image = cv2.hconcat([cv2.vconcat([sourceimage,warpedimage]),targetimage])
        
        self.image_list.append(image)
        #####################################################################################################
        # Open viewer
        #####################################################################################################
#         manager = viz_utils.CustomDrawGeometryWithKeyCallback(
#             geometry_dict, alignment_dict, matches_dict
#         )
#         manager.custom_draw_geometry_with_key_callback()
        

In [31]:
RGBDVideoResults(datapath="../DeepDeform/new_test/",gridsize=0.0006)

Deformable Model: Initialization Complete, Node:89 Edges:44


NameError: name 'i' is not defined

In [ ]:
"../DeepDeform/new_test/"
sorted(os.listdir(datapath),key=lambda x: int(x.split('.')[0]))